In [1]:
import requests
import csv
from datetime import datetime, timedelta

url = "http://www.khoa.go.kr/api/oceangrid/tidalBuTemp/search.do"
params = {
    "ServiceKey": "Sfb4fiGk1sy9O8Qfnn4A==",
    "ObsCode": "KG_0028",
    "Date": "",
    "ResultType": "json"
}

start_date = datetime(2013, 1, 1).date()  # Set the starting date
num_days = 3650

# Open the CSV file for writing
with open("data.csv", "w", newline="") as file:
    writer = csv.writer(file)

    for i in range(num_days):
        current_date = start_date + timedelta(days=i)
        current_date_str = current_date.strftime("%Y%m%d")
        params["Date"] = current_date_str

        response = requests.get(url, params=params)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            # Process the retrieved JSON data as per your requirements
            # You can access the specific data elements using dictionary keys

            # Example: Write data to CSV file
            writer.writerow([current_date_str, data])  # Customize the columns as needed
        else:
            # Handle the error case
            print(f"Error for {current_date_str}:", response.status_code)

In [42]:
import pandas as pd
import numpy as np

# Define the column names
# column_names = ['Column1', 'Column2']

# Read the CSV file and specify the column names
df = pd.read_csv('data_03.csv', index_col=0, parse_dates=True)
# df= df.set_index(pd.to_datetime(df.1))
# Display the entire DataFrame
df['평균수온'] = df.mean(axis=1)
df = df[['평균수온']]

df.loc[df['평균수온'] == 0, '평균수온'] = np.nan

df = df.dropna()
df

# # Display the first few rows
# print(df.head())


,평균수온
일시,
2013-01-01,15.410204
2013-01-02,15.279167
2013-01-03,15.121739
2013-01-04,15.091667
2013-01-05,15.040816
...,...
2022-12-25,16.727755
2022-12-26,16.173673
2022-12-27,15.887347
